In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.6 MB/s eta 0:00:00


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
import kerastuner as kt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
import seaborn as sns
import matplotlib.pyplot as plt

#  Import and read the heart_disease_dataset.csv.
import pandas as pd

url= 'https://raw.githubusercontent.com/Nathanhans/project-4/main/heart_disease_dataset.csv'
heart_disease_df = pd.read_csv(url)
heart_disease_df.head()

<ipython-input-2-15bb455f3564>:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,...,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,Alabama,Female,Very good,4,0,Within past year (anytime less than 12 months ...,Yes,9,None of them,No,...,1.60,71.67,27.99,No,No,Yes,Yes,"Yes, received Tdap",No,No
1,Alabama,Male,Very good,0,0,Within past year (anytime less than 12 months ...,Yes,6,None of them,No,...,1.78,95.25,30.13,No,No,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,No
2,Alabama,Male,Very good,0,0,Within past year (anytime less than 12 months ...,No,8,"6 or more, but not all",No,...,1.85,108.86,31.66,Yes,No,No,Yes,"No, did not receive any tetanus shot in the pa...",No,Yes
3,Alabama,Female,Fair,5,0,Within past year (anytime less than 12 months ...,Yes,9,None of them,No,...,1.70,90.72,31.32,No,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,Yes
4,Alabama,Female,Good,3,15,Within past year (anytime less than 12 months ...,Yes,5,1 to 5,No,...,1.55,79.38,33.07,No,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,No


In [3]:
#confirming lenght to make sure no data was lost
len(heart_disease_df)

246022

In [4]:
# Drop the non-beneficial columns such as State
heart_disease_df.head()

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,...,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,Alabama,Female,Very good,4,0,Within past year (anytime less than 12 months ...,Yes,9,None of them,No,...,1.60,71.67,27.99,No,No,Yes,Yes,"Yes, received Tdap",No,No
1,Alabama,Male,Very good,0,0,Within past year (anytime less than 12 months ...,Yes,6,None of them,No,...,1.78,95.25,30.13,No,No,Yes,Yes,"Yes, received tetanus shot but not sure what type",No,No
2,Alabama,Male,Very good,0,0,Within past year (anytime less than 12 months ...,No,8,"6 or more, but not all",No,...,1.85,108.86,31.66,Yes,No,No,Yes,"No, did not receive any tetanus shot in the pa...",No,Yes
3,Alabama,Female,Fair,5,0,Within past year (anytime less than 12 months ...,Yes,9,None of them,No,...,1.70,90.72,31.32,No,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,Yes
4,Alabama,Female,Good,3,15,Within past year (anytime less than 12 months ...,Yes,5,1 to 5,No,...,1.55,79.38,33.07,No,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,No


In [5]:
# nuniuqe to view the unique data in each column
heart_disease_df.nunique()

State                          54
Sex                             2
GeneralHealth                   5
PhysicalHealthDays             31
MentalHealthDays               31
LastCheckupTime                 4
PhysicalActivities              2
SleepHours                     23
RemovedTeeth                    4
HadHeartAttack                  2
HadAngina                       2
HadStroke                       2
HadAsthma                       2
HadSkinCancer                   2
HadCOPD                         2
HadDepressiveDisorder           2
HadKidneyDisease                2
HadArthritis                    2
HadDiabetes                     4
DeafOrHardOfHearing             2
BlindOrVisionDifficulty         2
DifficultyConcentrating         2
DifficultyWalking               2
DifficultyDressingBathing       2
DifficultyErrands               2
SmokerStatus                    4
ECigaretteUsage                 4
ChestScan                       2
RaceEthnicityCategory           5
AgeCategory   

In [6]:
heart_disease_df= heart_disease_df.copy().drop(['State'],axis=1)

In [7]:
target_column = heart_disease_df['HadHeartAttack']
df = heart_disease_df.drop('HadHeartAttack', axis=1)  # Drop the target column from the DataFrame

# Apply get_dummies() to the remaining DataFrame
df_dummies = pd.get_dummies(df)

# Concatenate the target column with the DataFrame of dummy variables
df_processed = pd.concat([target_column, df_dummies], axis=1)


In [8]:
df_processed.head()

,HadHeartAttack,PhysicalHealthDays,MentalHealthDays,SleepHours,HeightInMeters,WeightInKilograms,BMI,Sex_Female,Sex_Male,GeneralHealth_Excellent,...,PneumoVaxEver_Yes,"TetanusLast10Tdap_No, did not receive any tetanus shot in the past 10 years","TetanusLast10Tdap_Yes, received Tdap","TetanusLast10Tdap_Yes, received tetanus shot but not sure what type","TetanusLast10Tdap_Yes, received tetanus shot, but not Tdap",HighRiskLastYear_No,HighRiskLastYear_Yes,CovidPos_No,CovidPos_Tested positive using home test without a health professional,CovidPos_Yes
0,No,4,0,9,1.60,71.67,27.99,1,0,0,...,1,0,1,0,0,1,0,1,0,0
1,No,0,0,6,1.78,95.25,30.13,0,1,0,...,1,0,0,1,0,1,0,1,0,0
2,No,0,0,8,1.85,108.86,31.66,0,1,0,...,1,1,0,0,0,1,0,0,0,1
3,No,5,0,9,1.70,90.72,31.32,1,0,0,...,1,1,0,0,0,1,0,0,0,1
4,No,3,15,5,1.55,79.38,33.07,1,0,0,...,1,1,0,0,0,1,0,1,0,0


In [9]:
# Split our preprocessed data into our features and target arrays
y = df_processed['HadHeartAttack']
X = df_processed.drop(['HadHeartAttack'], axis = 1)


In [10]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)

(184516, 100)
(61506, 100)


In [13]:
# Create instance of labelencoder
le= LabelEncoder()
# Fit the LabelEncoder on the target variable
y_train_encoded = le.fit_transform(y_train)
#Fit the LabelEncoder on the test variable
y_test_encoded= le.fit_transform(y_test)

In [14]:
print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"y_train shape: {y_train_encoded.shape}")
print(f"X_train_scaled data type: {X_train_scaled.dtype}")
print(f"y_train data type: {y_train_encoded.dtype}")

X_train_scaled shape: (184516, 100)
y_train shape: (184516,)
X_train_scaled data type: float64
y_train data type: int64


**Model Architecture 3**:

Input Layer: Number of Input Features (100)

Hidden Layer 1: 9 nodes, Activation Function: relu

Output Layer: 1 node, Activation Function: sigmoid

Number of epochs: 7



In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer ()
nn.add(tf.keras.layers.Dense(units=9, activation="relu", input_dim=100))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 909       
                                                                 
 dense_1 (Dense)             (None, 1)                 10        
                                                                 
Total params: 919 (3.59 KB)
Trainable params: 919 (3.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train_encoded, epochs=7)

Epoch 1/7
5767/5767 [==============================] - 12s 2ms/step - loss: 0.1694 - accuracy: 0.9405
Epoch 2/7
5767/5767 [==============================] - 12s 2ms/step - loss: 0.1458 - accuracy: 0.9484
Epoch 3/7
5767/5767 [==============================] - 12s 2ms/step - loss: 0.1445 - accuracy: 0.9490
Epoch 4/7
5767/5767 [==============================] - 12s 2ms/step - loss: 0.1440 - accuracy: 0.9486
Epoch 5/7
5767/5767 [==============================] - 12s 2ms/step - loss: 0.1435 - accuracy: 0.9493
Epoch 6/7
5767/5767 [==============================] - 11s 2ms/step - loss: 0.1432 - accuracy: 0.9491
Epoch 7/7
5767/5767 [==============================] - 11s 2ms/step - loss: 0.1430 - accuracy: 0.9495


In [18]:
predictions= nn.predict(X_test_scaled)
predictions

1923/1923 [==============================] - 4s 2ms/step


array([[0.00103274],
       [0.18590705],
       [0.03177038],
       ...,
       [0.000776  ],
       [0.0021359 ],
       [0.0507631 ]], dtype=float32)

In [19]:
round_array = np.array([int(np.round(i)) for i in predictions])
round_array

<ipython-input-19-643e8450219d>:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  round_array = np.array([int(np.round(i)) for i in predictions])


array([0, 0, 0, ..., 0, 0, 0])

In [20]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test_encoded, round_array)

array([[57528,   619],
       [ 2543,   816]])

In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_test_encoded, round_array))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97     58147
           1       0.57      0.24      0.34      3359

    accuracy                           0.95     61506
   macro avg       0.76      0.62      0.66     61506
weighted avg       0.94      0.95      0.94     61506



In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test_encoded,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1923/1923 - 2s - loss: 0.1444 - accuracy: 0.9486 - 2s/epoch - 1ms/step
Loss: 0.1443832367658615, Accuracy: 0.9485903978347778


In [23]:
# Export our model to HDF5 file
from google.colab import files

nn.save("/content/sally_auto_opt_colab_hyper_param_model_3_updated.h5")
files.download("/content/sally_auto_opt_colab_hyper_param_model_3_updated.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>